In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
from IPython.display import display, Image, clear_output
clear_output()

In [ ]:
# creating, training, and deploying machine learning models
# with a no-code interface and deep learning framework support

!pip install ultralytics==8.0.196
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import ultralytics

In [ ]:
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/107.7 GB disk)


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="N1N7Zb4O1y8Rpkdnlyhn")
project = rf.workspace("2dod").project("defect-ee4bu")
version = project.version(3)
dataset = version.download("yolov8-obb")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Defect-3 in yolov8-obb:: 100%|██████████| 21246/21246 [00:04<00:00, 5042.67it/s]


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8m.pt data=/content/datasets/Defect-3/data.yaml epochs=50 batch=64 imgsz=640 plots=True save=True optimizer=SGD dropout=0.2 augment=True

/content
New https://pypi.org/project/ultralytics/8.2.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/Defect-3/data.yaml, epochs=50, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augme

In [ ]:
import subprocess
import random
import yaml
from ultralytics import YOLO
import os

def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def train_and_evaluate(weight_accuracy, weight_generalizability, hyperparams):
    command = (
        f"yolo task=detect mode=train model=yolov8m.pt data=/content/datasets/Defect-3/data.yaml "
        f"epochs=10 batch={hyperparams['batch']} imgsz=640 optimizer={hyperparams['optimizer']} "
        f"dropout={hyperparams['dropout']} augment=True"
    )

    # Run the training command and capture output
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        print(result.stdout)
        print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error during training: {e.stderr}")
        return None, hyperparams, None, None

    # Check if the weights file exists
    weights_path = f"{HOME}/runs/detect/train/weights/best.pt"
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"No such file: '{weights_path}'")

    # Load model and data
    model = YOLO(weights_path)
    data_yaml = load_yaml(f"{dataset.location}/data.yaml")
    val_data = data_yaml['val']  # Assuming 'val' key contains validation data path

    # Evaluate the model on validation data
    results = model.predict(val_data)
    accuracy = results.metric('accuracy')  # Replace with actual metric function
    generalizability = 1 - results.metric('overfit_metric')  # Hypothetical example

    # Custom objective function
    objective = weight_accuracy * accuracy + weight_generalizability * generalizability
    return objective, hyperparams, accuracy, generalizability

# Define hyperparameter search space
hyperparam_space = {
    'optimizer': ['SGD', 'Adam'],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'batch': [16, 32, 64]
}

# Scenarios
scenarios = [
    (0.6, 0.4),
    (0.5, 0.5),
    (0.9, 0.1),
]

results = []

for weight_accuracy, weight_generalizability in scenarios:
    # Randomly sample hyperparameters for each scenario
    hyperparams = {key: random.choice(values) for key, values in hyperparam_space.items()}
    objective, hyperparams, accuracy, generalizability = train_and_evaluate(weight_accuracy, weight_generalizability, hyperparams)
    if objective is not None:
        results.append({
            'weight_accuracy': weight_accuracy,
            'weight_generalizability': weight_generalizability,
            'objective': objective,
            'hyperparams': hyperparams,
            'accuracy': accuracy,
            'generalizability': generalizability,
        })

# Generate report
for result in results:
    print(f"Scenario - Weight Accuracy: {result['weight_accuracy']}, Weight Generalizability: {result['weight_generalizability']}")
    print(f"Best Hyperparameters: {result['hyperparams']}")
    print(f"Objective Score: {result['objective']}")
    print(f"Accuracy: {result['accuracy']}")
    print(f"Generalizability: {result['generalizability']}\n")


Error during training: New https://pypi.org/project/ultralytics/8.2.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/Defect-3/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visua